## Домашнє завдання №5
## Класифікування активності за даними акселерометра

### 1. Отримання датасету

In [141]:
mypath = '.\data'

In [142]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd

In [143]:
# отримаємо список підкаталогів в теці "data"
activites = listdir(mypath)
activites

['idle', 'running', 'stairs', 'walking']

In [144]:
# Виведемо кількість файлів в кожній теці активності
for act in activites:
    path = join(mypath, act)  # створення шляху для активності
    frames = listdir(path)  # отримання списку файлів в 'path'
    print(act, ': ', len(frames))  # виведення кількості файлів в 'act'

idle :  1039
running :  3408
stairs :  165
walking :  1850


In [145]:
# Виведемо для прикладу зміст шостого файлу з теки 'walking' ('сирі' дані)
frames = listdir(join(mypath, 'walking'))
frame = pd.read_csv(join(join(mypath, 'walking'),frames[5]))
frame  # датасет містить посекундні дані переміщення, кожний рядок - одна секунда

,accelerometer_X,accelerometer_Y,accelerometer_Z
0,-4.352658,-9.327810,0.713472
1,-3.955221,-16.496050,3.438074
2,-2.705448,-7.436390,1.383849
3,-4.577713,-17.645266,1.944092
4,7.192182,-25.201366,-16.903065
5,1.575385,-10.620679,1.589750
6,-0.028730,-5.425261,0.823605
7,3.418920,-6.675034,1.158794
8,1.891419,-11.669339,-1.570596
9,-1.647211,-5.305551,9.169792


## 2. Підготовка даних

In [146]:
# Імпортуємо модуль для оцінювання середньоквадратичної помилки "mean_squared_error" та
# модуль для обчислення середньої абсолютної помилки "mean_absolute_error".
# Ці метрики використовують для вимірювання різниці між прогнозованими та фактичними значеннями.
from sklearn.metrics import mean_squared_error, mean_absolute_error
# Імпортуємо модуль для перетворення Фур'є, тобто він перетворює дані з часової області в частотну.
# "fft" - для комплексних даних (містять дійсну та уявну частину), "rfft" - для дійсних
from scipy.fftpack import fft, rfft
# Імпортуємо модуль для первинної обробки даних
from sklearn import preprocessing

In [147]:
# Створимо список часових ознак для датасету (з лекційного матеріалу)
features = [
    'skew_X',  # асиметрія розподілу даних
    'skew_Y',
    'skew_Z',
    'kurtosis_X',  # гострота пику розподілу даних
    'kurtosis_Y',
    'kurtosis_Z',
    'max_X',
    'max_Y',
    'max_Z',
    'min_X',
    'min_Y',
    'min_Z',
    'mean_X',
    'mean_Y',
    'mean_Z',
    'std_X',  # стандартне відхилення, що являється мірою розкиду даних
    'std_Y',
    'std_Z',
    'variance_X',  # дисперсія (розкид даних)
    'variance_Y',
    'variance_Z',
    'median_X',
    'median_Y',
    'median_Z',
    'index_max_X',
    'index_max_Y',
    'index_max_Z',
    'index_min_X',
    'index_min_Y',
    'index_min_Z',
    'corelation_X_Y',
    'corelation_X_Z',
    'corelation_Y_Z',
    'mae_X',  # середня абсолютна різниця між значеннями даних та прогнозними значеннями
    'mae_Y',
    'mae_Z',
    'rmse_X',  # середньоквадратична різниця між значеннями даних та прогнозними значеннями
    'rmse_Y',
    'rmse_Z'
    ]

len(features)

39

In [148]:
# Створимо ф-ю, яка рахуватиме всі часові ознаки
def get_stat_features(frame):

    features = []
    features = np.array(features)  # перетворення порожнього списку в массив NumPy
    features = np.concatenate((features, frame.skew(axis=0).values), axis=0)
    # конкатенація масиву 'features' з масивом ознак, обчислених з використанням методу 'skew' (асиметрія) для DataFrame 'frame'
    # 'axis=0' - конкатенація виконується по стовпчикам (по вертикалі)
    # '.values' - перетворює результати в массив NumPy
    # 'np.concatenate' - додавання стовпців до масиву
    features = np.concatenate((features, frame.kurt(axis=0).values), axis=0)
    # '.kurt' - kurtosis, розрахунок ексцеса (гострота пику розподілу даних)
    features = np.concatenate((features, frame.max(axis=0).values), axis=0)
    features = np.concatenate((features, frame.min(axis=0).values), axis=0)
    features = np.concatenate((features, frame.mean(axis=0).values), axis=0)
    features = np.concatenate((features, frame.std(axis=0).values), axis=0)
    features = np.concatenate((features, frame.var(axis=0).values), axis=0)
    features = np.concatenate((features, frame.median(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmax(axis=0).values), axis=0)
    features = np.concatenate((features, frame.idxmin(axis=0).values), axis=0)

    # створення матриці кореляції для всіх пар колонок
    correlations = frame.corr()  # розрахунок коефіцієнтів кореляції для кожної пари колонок
    # створення матриці з конкретними значеннями кореляції (на базі матриці "correlations")
    corr = np.array([
        correlations['accelerometer_X']['accelerometer_Y'],
        correlations['accelerometer_X']['accelerometer_Z'],
        correlations['accelerometer_Y']['accelerometer_Z']
    ])

    # додамо стовпчики з коефіцієнтами кореляції до матриці features
    features = np.concatenate((features, corr), axis=0)

    # створення нових колонок у "frame"
    frame['mean_X'] = frame.mean(axis=0)['accelerometer_X']  # міститиме середнє знач. для даних в колонці 'accelerometer_X'
    frame['mean_Y'] = frame.mean(axis=0)['accelerometer_Y']
    frame['mean_Z'] = frame.mean(axis=0)['accelerometer_Z']
    
    # розрахуємо середню абсолютну помилку (MAE) між двома стовпчиками
    mae_X = mean_absolute_error(frame['accelerometer_X'], frame['mean_X'])
    mae_Y = mean_absolute_error(frame['accelerometer_Y'], frame['mean_Y'])
    mae_Z = mean_absolute_error(frame['accelerometer_Z'], frame['mean_Z'])

    # розрахуємо корінь середньоквадратичної помилки (RMSE) між двома стовпчиками
    rmse_x = np.sqrt(mean_squared_error(frame['accelerometer_X'], frame['mean_X']))
    rmse_y = np.sqrt(mean_squared_error(frame['accelerometer_Y'], frame['mean_Y']))
    rmse_z = np.sqrt(mean_squared_error(frame['accelerometer_Z'], frame['mean_Z']))

    # створення масиву
    metrics = np.array([mae_X, mae_Y, mae_Z, rmse_x, rmse_y, rmse_z])
    # поєднання масивів
    features = np.concatenate((features, metrics), axis=0)

    return features

In [149]:
len(get_stat_features(frame))

39

In [150]:
# Створимо ф-ю, яка рахуватиме коефіцієнти швидкого перетворення Фур'є та нормалізуватиме їх
def get_fft_features(frame):
    
    fft_X = np.array(preprocessing.normalize([rfft(frame['accelerometer_X'].values)])[0])
    fft_Y = np.array(preprocessing.normalize([rfft(frame['accelerometer_Y'].values)])[0])
    fft_Z = np.array(preprocessing.normalize([rfft(frame['accelerometer_Z'].values)])[0])

    features = np.concatenate((fft_X, fft_Y), axis=0)
    features = np.concatenate((features, fft_Z), axis=0)

    return features

In [151]:
len(get_stat_features(frame))

69

#### Перевірка працездатності функціоналу ф-ї get_stat_features (чернетка)

In [152]:
frame.skew(axis=0)  # розрахунок асиметрії даних для кожного стовпця 'frame'

accelerometer_X   -0.323847
accelerometer_Y    0.162237
accelerometer_Z    2.061646
mean_X             0.000000
mean_Y             0.000000
mean_Z             0.000000
dtype: float64

In [153]:
frame.kurt(axis=0)  # розрахунок ексцеса для кожного стовпця 'frame'

accelerometer_X     0.134180
accelerometer_Y     0.542627
accelerometer_Z    10.326116
mean_X              0.000000
mean_Y              0.000000
mean_Z              0.000000
dtype: float64

In [154]:
frame.max(axis=0)  # розрахунок максимального значення для кожного стовпця 'frame'

accelerometer_X    11.113884
accelerometer_Y     6.435614
accelerometer_Z    32.599450
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [155]:
frame.min(axis=0)  # розрахунок мінімального значення для кожного стовпця 'frame'

accelerometer_X   -12.277467
accelerometer_Y   -25.201366
accelerometer_Z   -16.903065
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [156]:
frame.mean(axis=0)  # розрахунок середнього значення для кожного стовпця 'frame'

accelerometer_X     0.285548
accelerometer_Y   -10.916123
accelerometer_Z     0.366473
mean_X              0.285548
mean_Y            -10.916123
mean_Z              0.366473
dtype: float64

In [157]:
frame.std(axis=0)  # розрахунок стандартного відхилення для кожного стовпця 'frame'

accelerometer_X    5.090906e+00
accelerometer_Y    6.820465e+00
accelerometer_Z    7.811986e+00
mean_X             5.646013e-17
mean_Y             3.613448e-15
mean_Z             1.129203e-16
dtype: float64

In [158]:
frame.var(axis=0)  # розрахунок дисперсії (розкид даних) для кожного стовпця 'frame'

accelerometer_X    2.591732e+01
accelerometer_Y    4.651874e+01
accelerometer_Z    6.102712e+01
mean_X             3.187746e-33
mean_Y             1.305701e-29
mean_Z             1.275098e-32
dtype: float64

In [159]:
frame.idxmax(axis=0)  #  індекс першого входження максимального значення для кожного стовпця 'frame'

accelerometer_X    18
accelerometer_Y    25
accelerometer_Z    11
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

In [160]:
frame.idxmin(axis=0)  #  індекс першого входження мінімального значення для кожного стовпця 'frame'

accelerometer_X    25
accelerometer_Y     4
accelerometer_Z     4
mean_X              0
mean_Y              0
mean_Z              0
dtype: int64

#### Функції для створення масивів NumPy з різними даними

In [161]:
# Створимо першу ф-ю, яка виконуватиме підготовку даних
def class_data_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з даними із датасету;
    у - з мітками ("class_number")
    """

    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується 
        frame = frame.values  # масив DataFrame перетворюється в масив NumPy для зручності
        frame = frame.flatten()  # масив даних перетворюється на одновимірний масив (flatten)
        X.append(frame)  # одновимірний масив додається в список 'X'
    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [162]:
# Створимо другу ф-ю, яка виконуватиме підготовку даних
def class_data_stat_fft_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету часовими ознаками та нормалізованими коефіцієнтами швидкого перетворення Фур'є;
    у - з мітками ("class_number")
    """

    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features_stat = get_stat_features(frame)  # розрахунок всіх часових ознак
        features_fft = get_fft_features(frame)  # розрахунок коефіцієнтів швидкого перетворення Фур'є та їх нормалізація
        features = np.concatenate((features_stat, features_fft), axis=0)
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [163]:
# Створимо третю ф-ю, яка виконуватиме підготовку даних
def class_data_fft_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету нормалізованими коефіцієнтами швидкого перетворення Фур'є;
    у - з мітками ("class_number")
    """
    
    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features = get_fft_features(frame)  # розрахунок коефіцієнтів швидкого перетворення Фур'є та їх нормалізація
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

In [164]:
# Створимо четверту ф-ю, яка виконуватиме підготовку даних
def class_data_stat_prepare(class_name, class_number):
    """
    Створення двох масивів NumPy: 
    Х - з розрахованими на основі датасету часовими ознаками;
    у - з мітками ("class_number")
    """
    
    path = join(mypath, class_name)  # формування шляху до директорії певного класу даних
    X = []
    for item in listdir(path):  # ітерація по файлам в "path"
        frame = pd.read_csv(join(path, item)) # кожний файл зчитується
        features_stat = get_stat_features(frame)  # розрахунок всіх часових ознак
        X.append(features)

    y = [class_number]*len(X)  # створення списку однакових значень "class_number" довжиною "len(X)"
    X = np.array(X)  # перетворення списку в масив NumPy
    y = np.array(y)  # перетворення списку в масив NumPy
    return X, y

#### Створення датасету

In [165]:
# Створимо ф-ю, яка створюватиме масив даних та масив міток (номерів класів даних - від 0 до 3), використовуючи одну з 4х вище написаних ф-й
def create_dataset(class_prepare):

    X_idle, y_idle = class_prepare('idle', 0)
    X_walking, y_walking = class_prepare('walking', 1)
    X_stairs, y_stairs = class_prepare('stairs', 2)
    X_running, y_running = class_prepare('running', 3)

    X = np.concatenate((X_idle, X_walking), axis=0)
    X = np.concatenate((X, X_stairs), axis=0)
    X = np.concatenate((X, X_running), axis=0)

    Y = np.concatenate((y_idle, y_walking))
    Y = np.concatenate((Y, y_stairs))
    Y = np.concatenate((Y, y_running))

    return X, Y

In [166]:
# Створимо масив даних но основі часових ознак та з номерними мітками
X_stat, Y_stat = create_dataset(class_data_stat_prepare)

# Запишемо отримані дані у файли в бінарному форматі
import pickle
with open('stat_data_X.pickle', 'wb') as f:
    pickle.dump(X_stat, f)

with open('stat_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_stat, f)

In [167]:
# Створимо масив даних но основі перетворення Фур'є та з номерними мітками
X_fft, Y_fft = create_dataset(class_data_fft_prepare)

# Запишемо отримані дані у файли в бінарному форматі
with open('fft_data_X.pickle', 'wb') as f:
    pickle.dump(X_fft, f)

with open('fft_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_fft, f)

In [168]:
# Створимо масив даних на основі часових ознак та номерних міток
X_fft_stat, Y_fft_stat = create_dataset(class_data_stat_fft_prepare)

# Запишемо отримані дані у файли в бінарному форматі
with open('fft_stat_data_X.pickle', 'wb') as f:
    pickle.dump(X_fft_stat, f)

with open('fft_stat_data_Y.pickle', 'wb') as f:
    pickle.dump(Y_fft_stat, f)

#### Завантаження датасету

In [169]:
# Ф-я завантаження датасету. Приймає аргумент: 'fft', 'stat' або 'fft_stat'
def open_data_set(flag):
    name = flag + '_data_'
    
    with open(name + 'X.pickle', 'rb') as f:
        X = pickle.load(f)
    with open(name + 'Y.pickle', 'rb') as f:
        Y = pickle.load(f)
    return X, Y

In [170]:
# Завантажимо датасет 'fft_data_Y.pickle'
X, Y = open_data_set('fft')
# Перевірка коректності роботи завантажувача pickle
np.array_equal(X, X_fft)  # перевірка однаковості

True

## 3. Класифікація методом опорних векторів SVM (Support Vector Machine)

In [171]:
from sklearn.model_selection import cross_val_score  # оцінювання продуктивності моделі з використанням крос-валідації
from sklearn.metrics import f1_score, accuracy_score  # розрахунок метрик якості моделі
from sklearn import svm

In [172]:
# Завантажимо датасет 'fft_data_Y.pickle'
X, Y = open_data_set('fft')

# Розіб'ємо завантажені дані на навчальну і тестову частини
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3)

### 3.1. SVM gamma=0.01

In [173]:
# Створимо класифікатор для багатокласової класифікації з gamma=0.01
classifier = svm.SVC(decision_function_shape='ovo', kernel='rbf', gamma=0.01, probability=True)
# 'svm.SVC' - створення об'єкта класифікатора
# "decision_function_shape='ovo'" - використовується метод "one-vs-one"
# "kernel='rbf'" - задає використання радіально-базової функції (Radial Basis Function, RBF) як ядро SVM
# "gamma=0.01" - параметр, який впливає на форму гіперповерхні рішення
# "probability=True" - вказує, що класифікатор повинен надавати оцінки ймовірності кожного класу при виклику методу predict_proba

In [174]:
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', gamma=0.01, probability=True)

In [175]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()  # середнє значення продуктивності

0.9721429408662015

In [176]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9804022692109334

In [177]:
# Створимо ф-ю для підрахунку кількості індексів класів
def make_report(y_test_proba, y_test):

    idle_test = 0
    walking_test = 0
    stairs_test = 0
    running_test = 0

    for i in y_test_proba:
        if i == 0:
            idle_test += 1
        elif i == 1:
            walking_test += 1
        elif i == 2:
            stairs_test += 1
        elif i == 3:
            running_test += 1

    idle_pred = 0
    walking_pred = 0
    stairs_pred = 0
    running_pred = 0

    for i in y_test_proba:
        if i == 0:
            idle_pred += 1
        elif i == 1:
            walking_pred += 1
        elif i == 2:
            stairs_pred += 1
        elif i == 3:
            running_pred += 1

    print(f'idle test -- {idle_test} pred -- {idle_pred}')
    print(f'walking test -- {walking_test} pred -- {walking_pred}')
    print(f'stairs test -- {stairs_test} pred -- {stairs_pred}')
    print(f'running test -- {running_test} pred -- {running_pred}')

In [178]:
make_report(y_test_proba, y_test)

idle test -- 304 pred -- 304
walking test -- 589 pred -- 589
stairs test -- 29 pred -- 29
running test -- 1017 pred -- 1017


### 3.2. SVM gamma='scale'

In [179]:
# Створимо класифікатор для багатокласової класифікації з gamma='scale' (параметр за замовчуванням) (атвоматичне масштабування gamma) 
classifier = svm.SVC(decision_function_shape='ovo', probability=True)
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', probability=True)

In [180]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9778892926214617

In [181]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9845281072717896

In [182]:
make_report(y_test_proba, y_test)

idle test -- 302 pred -- 302
walking test -- 587 pred -- 587
stairs test -- 31 pred -- 31
running test -- 1019 pred -- 1019


### 3.3. SVM kernel='linear'

In [184]:
# Створимо класифікатор для багатокласової класифікації з ядром kernel='linear' 
classifier = svm.SVC(decision_function_shape='ovo', probability=True, kernel='linear')
# Проведемо навчання класифікатора на навчальній вибірці підготовлених даних но основі часових ознак
classifier.fit(X_train, y_train)

SVC(decision_function_shape='ovo', kernel='linear', probability=True)

In [185]:
# Оцінимо продуктивність моделі з використанням крос-валідації
scores = cross_val_score(classifier, X_train, y_train, cv=10)  # cv=10 - к-ть фолдів (10-ти кратна крос-валідація)
scores.mean()

0.9767865166344333

In [186]:
# Перевіримо точність передбачень на тестовій вибірці
y_test_proba = classifier.predict_proba(X_test)  # ймовірність приналежності до класів для кожного об'єкта у тестовому наборі
y_test_proba = np.argmax(y_test_proba, axis=1)  # вибір індексу з найвищою ймовірністю (тобто класу з найвищою ймовірністю) для кожного об'єкта у тестовому наборі
accuracy = accuracy_score(y_test, y_test_proba)  # порівнює фактичні мітки (y_test) з передбаченими мітками (y_test_proba) і обчислює точність передбачення
accuracy

0.9819494584837545

In [187]:
make_report(y_test_proba, y_test)

idle test -- 302 pred -- 302
walking test -- 594 pred -- 594
stairs test -- 24 pred -- 24
running test -- 1019 pred -- 1019


### 3.4. SVM OVR (One-Vs-Rest)